In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [12]:
concept_model = tf.keras.models.load_model("sensor_data_to_concept_model.keras")
activity_model = tf.keras.models.load_model("concepts_to_true_labels_model.keras")

In [13]:
df_sensor = pd.read_csv('./data/final_dataset.csv')
df_windows = pd.read_csv('./data/final_window_labels.csv') 

In [14]:
def extract_window_robust(df_sensor, window_row, time_tolerance=0.5):
    """
    Extract sensor data with time tolerance to handle mismatches.
    """
    user = window_row['user']
    activity = window_row['activity']
    start_time = window_row['start_time']
    end_time = window_row['end_time']
    
    # Get data for this user/activity
    user_activity_data = df_sensor[(df_sensor['user'] == user) & 
                                  (df_sensor['activity'] == activity)].copy()
    
    if len(user_activity_data) == 0:
        return None
    
    # Find data within time window with tolerance
    mask = ((user_activity_data['time_s'] >= start_time - time_tolerance) & 
            (user_activity_data['time_s'] <= end_time + time_tolerance))
    
    window_data = user_activity_data[mask]
    
    if len(window_data) < 10:  # Need minimum samples
        return None
    
    # Extract sensor readings
    sensor_data = window_data[['x-axis', 'y-axis', 'z-axis']].values
    
    # Pad or truncate to fixed length (e.g., 100 samples)
    target_length = 100
    if len(sensor_data) > target_length:
        # Randomly sample if too long
        indices = np.random.choice(len(sensor_data), target_length, replace=False)
        sensor_data = sensor_data[indices]
    elif len(sensor_data) < target_length:
        # Pad with last value if too short
        padding = np.tile(sensor_data[-1:], (target_length - len(sensor_data), 1))
        sensor_data = np.vstack([sensor_data, padding])
    
    return sensor_data

def extract_windows_robust(df_sensor, df_windows):
    """
    Extract all windows with robust time matching for 6 concepts.
    """
    X = []
    y_p = []  # periodicity
    y_t = []  # temporal_stability
    y_c = []  # coordination
    y_mi = [] # motion_intensity
    y_vd = [] # vertical_dominance
    y_sp = [] # static_posture
    
    for _, window_row in df_windows.iterrows():
        window_data = extract_window_robust(df_sensor, window_row)
        if window_data is not None:
            X.append(window_data)
            y_p.append(window_row['periodicity'])
            y_t.append(window_row['temporal_stability'])
            y_c.append(window_row['coordination'])
            y_mi.append(window_row['motion_intensity'])
            y_vd.append(window_row['vertical_dominance'])
            y_sp.append(window_row['static_posture'])
    
    return np.array(X), np.array(y_p), np.array(y_t), np.array(y_c), np.array(y_mi), np.array(y_vd), np.array(y_sp)

In [ ]:
X_for_activity, _, _, _, _, _, _ = extract_windows_robust(df_sensor, df_windows)

In [21]:
predictions = concept_model.predict(X_for_activity)

periodicity_pred = np.argmax(predictions[0], axis=1)
temporal_stability_pred = np.argmax(predictions[1], axis=1)
coordination_pred = np.argmax(predictions[2], axis=1)
motion_intensity_pred = predictions[3].flatten()
vertical_dominance_pred = predictions[4].flatten()
static_posture_pred = np.argmax(predictions[5], axis=1)

pred_df = pd.DataFrame({
    'Periodicity': periodicity_pred // 2,
    'Temporal_Stability': temporal_stability_pred // 2,
    'Coordination': coordination_pred // 2,
    'Motion_Intensity': motion_intensity_pred // 2,
    'Vertical_Dominance': vertical_dominance_pred,
    'Static_Posture': static_posture_pred 
})

print("===First 10 predictions of the csv===")
print(pred_df.head(10))
print(pred_df.value_counts())

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
===First 10 predictions of the csv===
   Periodicity  Temporal_Stability  Coordination  Motion_Intensity  \
0            1                   0             0               0.0   
1            0                   0             0               0.0   
2            0                   0             0               0.0   
3            0                   0             0               0.0   
4            1                   0             0               0.0   
5            1                   0             0               0.0   
6            1                   0             0               0.0   
7            1                   1             1               0.0   
8            0                   1             1               0.0   
9            0                   1             1               0.0   

   Vertical_Dominance  Static_Posture  
0            0.229345               0  
1            0.211995               0  
2            0.159404            